In [ ]:
import numpy as np
import pandas as pd 

from fuzzyfication import Fuzzification
#from tnorm import tnorm

from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
from copy import deepcopy

from scipy.optimize import minimize

from basicfuzzy import trimf, trapmf

import scipy.io

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))
    print(len(A))

In [ ]:
#Functions for t-norm

def tnorm_product(values):
    return np.prod(values,axis=0)

def tnorm_minimum(values):
    return values.min(axis=0)

# Leitura de dados

In [ ]:
mat = scipy.io.loadmat('series/cluster4.mat')

In [ ]:
data = mat.get('cluster4')

In [ ]:
data = data[:,:4]

In [ ]:
#filedata = pd.read_csv("cluster1.csv", header=None)

In [ ]:
#data = filedata.values

In [ ]:
#data = data[:,:2]

In [ ]:
#data.shape

# Definição de parâmetros

In [ ]:
#Definicao de variaveis 

h_prev = 12
lag = 6                  #Actually, lag stands for all inputs for each serie. Example, lag = 2 uses s(t) and s(t-1) to predict s(t+1)
diff_series = False

#bin_values = 12; #Representação da binarização do tempo.
#lag_notused = []

num_series = data.shape[1]  #Numero de series do problema, extraído dos dados

max_rulesize = 5; #Max numbers of premises rules.
min_activation = 0.18 #Minimum activation

#####Definicao de funcoes######
#detrend_method = ''
#bin_method = ''

fuzzy_method = 'mfdef_cluster'
#Formacao de premissas

#form_method = 'form_NonExaustiveSparseBU_v4'
#form_param = []




# Pré-processamento de dados

In [ ]:
if diff_series:
    diff_data = data[1:,:] - data[0:data.shape[0]-1,:]

    in_sample = diff_data[:diff_data.shape[0]-h_prev,:]
    
    out_sample = diff_data[diff_data.shape[0]-h_prev:,:]
else:
    in_sample = data[:data.shape[0]-h_prev,:]
    out_sample = data[data.shape[0]-h_prev:,:]

In [ ]:
#Fazer detrend method

#Definicao do target
yt = np.zeros((in_sample.shape[0]-lag-1,num_series),dtype='float')

#Todas as entradas defasadas 
yp = np.zeros((in_sample.shape[0]-lag-1,num_series), dtype='float')
yp_lagged = np.zeros((in_sample.shape[0]-lag-1,num_series*lag),dtype='float')

In [ ]:
for i in range(num_series):
    yp[:,i] = in_sample[lag:in_sample.shape[0]-1,i]
    yt[:,i] = in_sample[lag+1:,i]
    for k in range(lag):
        yp_lagged[:,i*lag+k] = in_sample[lag-k:in_sample.shape[0]-k-1,i]
        #print(i*lag+k)


# Apenas para visualização de dados do problema

# Fuzzificação dos dados

In [ ]:
Fuzzyfy = Fuzzification(fuzzy_method)

In [ ]:
#Lembrete: 
#axis 0 - Registros da série
#axis 1 - Valor de pertinência ao conjunto Fuzzy
#axis 2 - Numero de séries

first_time = True
for n in range(num_series):
    
    _, mf_params = Fuzzyfy.fuzzify(in_sample[:,n],np.array([]))
    mX, _ = Fuzzyfy.fuzzify(yp[:,n],mf_params)
    mY, _ = Fuzzyfy.fuzzify(yt[:,n],mf_params)
    if first_time:
        mX_ = np.ndarray([mX.shape[0],mX.shape[1], num_series])
        mY_ = np.ndarray([mY.shape[0],mY.shape[1], num_series])
        mf_params_ = np.ndarray([mf_params.shape[0],num_series])
        first_time = False
    mX_[:,:,n] = mX
    mY_[:,:,n] = mY
    mf_params_[:,n] = mf_params
    #print(mf_params)
    #print(mX.shape)

In [ ]:
mX_lagged_ = np.ndarray([mX_.shape[0],mX_.shape[1],yp_lagged.shape[1]])
for i in range(num_series):
    mf_params = mf_params_[:,i]
    for j in range(lag):
        mX, _ = Fuzzyfy.fuzzify(yp_lagged[:,i*lag+j],mf_params)
        mX_lagged_[:,:,i*lag+j] = mX
        #print(i*lag+j)

In [ ]:
#Rules are created using t-norms, like "If x1 is A and x2 is B and x3 is C"
#Each rule will be stored as a list of tuples, whose each tuple corresponds to temporal series Number and fuzzy set
#Example: if we have only one rule as "if s1 is NB", then the fuzzy rule is stored as (1,0)
#This is just for antecedent rules. If we have a complete rule "If x1 is A and x2 is B and x3 is C then x4 is Aa",
#the last tuple will be always the consequent.






In [ ]:
#Here we use cardinality based on mean activation and relative frequency. We are going to use other functions to evaluate

def hybrid_func(values,min_act):
    values = np.asarray(values)
    if np.sum(values) == 0:
        return False, 0, 0
    else:
        mean_activation = np.mean(values[values>0],axis=0)
        freq = np.sum(values>0)
        freq_rel = freq/len(values)

        check_activation = mean_activation*freq_rel
        if check_activation > min_act:
            activation = True
        else:
            activation = False
        return activation, mean_activation, check_activation
    
def freq_func(values,min_act):
    values = np.asarray(values)
    if np.sum(values) == 0:
        return False, 0, 0
    else:
        freq = np.sum(values>0)
        freq_rel = freq/len(values)

        check_activation = freq_rel
        if check_activation > min_act:
            activation = True
        else:
            activation = False
        return activation, freq_rel, check_activation

def card_func(values,min_act):
    values = np.asarray(values)
    if np.sum(values) == 0:
        return False, 0, 0
    else:
        mean_activation = np.mean(values[values>0],axis=0)

        check_activation = mean_activation
        if check_activation > min_act:
            activation = True
        else:
            activation = False
        return activation, mean_activation, check_activation


In [ ]:
#Function to check if the antecedent is inside a function

def check_if_inside(val,eachRule):
    if val in eachRule:
        return True
    
    return False

In [ ]:
#Function just to sort antecedent rules in numerical order.

def rearranje_rules(rule):
    sorted_rule = []
    first_rules = np.unique([num_series[0] for num_series in rule])
    
    for val in first_rules:
        arranje_rule = [thisantecedent for thisantecedent in rule if thisantecedent[0] == val]
        arranje_rule.sort(key=lambda x: x[1])
        
        sorted_rule.extend(arranje_rule)
        
    return sorted_rule

In [ ]:
def check_duplicate_rules(val,rules):
    for rule in rules:
        if val == rule:
            return True
    return False

# Formulation

In [ ]:
rulesize = [0]

prem_terms = np.array([])

for r in range(0,max_rulesize):
    
    if r == 0:
        rules1 = []        
        for i in range(mX_lagged_.shape[2]):
            for j in range(mX_lagged_.shape[1]):
                #print(mX_lagged_[:,j,i].shape)
                activation, mean_activation, freq = card_func(mX_lagged_[:,j,i],min_activation)
                #print(mean_activation)
                #print(freq)
                #print(activation)
                if activation is True:
                    rules1.append([(i,j)])
                    if prem_terms.size == 0:
                        prem_terms = mX_lagged_[:,j,i]
                    else:
                        prem_terms = np.vstack((prem_terms,mX_lagged_[:,j,i]))
                    
        rules = np.empty(shape=[len(rules1), 1],dtype='object')
        rulesize.append(len(rules1))
        rules[:,0] = rules1
    
    else:
        lim_sup = rulesize[r]
        lim_inf = rulesize[r-1]
        new_rules = []           #Reinicia a lista de novas regras a cada layer de regra.
        #print(lim_sup,lim_inf)
        #Vamos verificar cada regra criada na rodada anterior. Para isso, verificamos o range que a lista de regras está alocada
        for rule in range(lim_inf,lim_inf+lim_sup):
            
            grow_rule = rules[rule,0]
            #print(grow_rule)
            for i in range(mX_lagged_.shape[2]):
                for j in range(mX_lagged_.shape[1]):
                    
                    
                    #Checa se o novo antecedente já está dentro do conjunto de antecedentes da regra
                    if check_if_inside((i,j),grow_rule):
                        continue
                    #Vamos concatenar todas as regras
                    count_tnorm = mX_lagged_[:,j,i]
                    
                    for r_size in grow_rule:               
                        count_tnorm = np.vstack((count_tnorm,mX_lagged_[:,r_size[1],r_size[0]]))
                        
                        #print(count_tnorm.shape)
                        #print(count_tnorm[:,1:4])
                    tnorm_ = tnorm_product(count_tnorm)
                    #print(tnorm_min[1:4])
                    activation, mean_activation, freq = card_func(tnorm_,min_activation)

                    if activation is True:
                        rule_to_append = deepcopy(grow_rule)
                        
                        rule_to_append.append((i,j))
                        #print(rule_to_append)
                        #print(mean_activation)
                        #print(freq)
                        sorted_rule = rearranje_rules(rule_to_append)
                        #print('Added {} to base rule'.format(sorted_rule))
                        print(sorted_rule) 
                        if not check_duplicate_rules(sorted_rule, new_rules):
                            new_rules.append(sorted_rule)
                        else:
                            print('Found one')
                            
                        prem_terms = np.vstack((prem_terms,tnorm_))

                        
        rulesize.append(len(new_rules))
        
        rules_ = np.empty(shape=[len(new_rules), 1],dtype='object')
        rules_[:,0] = new_rules
        rules = np.concatenate((rules,rules_))
                        
                    
        
        
        


# Now, split method

In [ ]:
#assert prem_terms.shape[1] == yt.shape[0]

In [ ]:
match_degree = np.ndarray(shape=(prem_terms.shape[0],mY_.shape[1],mY_.shape[2]))
complete_rules = np.empty(shape=[rules.shape[0], num_series],dtype='object')

for i in range(mY_.shape[2]):
    
    num_ = np.dot(prem_terms,mY_[:,:,i])
    
    ones = np.ones((num_.shape[0],num_.shape[1]))
    
    
    prem_den = np.sqrt(np.sum(prem_terms**2,axis=1))
    
    mY_den = np.sqrt(np.sum(mY_[:,:,i]**2,axis=0))
    
    
    den1 = ones*prem_den[:,None]
    den2 = (ones.T * mY_den[:,None]).T
    
    den_ = np.multiply(den1,den2)
    match_degree[:,:,i] = np.divide(num_,den_)
    
    best_match = np.argmax(match_degree[:,:,i],axis=1)
    
    for k in range(rules.shape[0]):
        one_rule = deepcopy(rules[k,0])
        one_rule.append((i,best_match[k]))
        complete_rules[k,i] = one_rule
        

In [ ]:
# complete_rules.shape

In [ ]:
#complete_rules[:,0]

# Reweighting

In [ ]:
count_tnorm.shape

In [ ]:
def find_rules_by_consequent(rules,n_serie,n_set):
    index_ = []
    k = 0
    for rule in rules[:,n_serie]:
        if rule[-1] == (n_serie,n_set):
            index_.append(k)
        k += 1
    #print(index_)
    return index_
    

In [ ]:
a = find_rules_by_consequent(complete_rules,1,4)

In [ ]:
wd_ = np.zeros(shape=(complete_rules.shape[0],mY_.shape[1],mY_.shape[2]))

global tryout
tryout = 0
def objective_function(x, mY, mX_a):
    global tryout
    #print(x)
    m_diag = np.diag(x)
    #print(x.shape)
    #print(mY.shape)
    #print(mX_a.shape)
    #print(m_diag.shape)
    a = np.sum(np.dot(m_diag,mX_a),axis=0)
    #print(a.shape)
    y = mY - a
    tryout += 1
    if (tryout%100 == 0):
        print('Attempt #{}'.format(tryout))
    return np.sum(np.sqrt(y**2))


def constraint_function(x):
    
    return np.sum(x) - 1


    
isEmpty = True

agg_training = np.zeros(shape=mY_.shape)

if isEmpty == True:
    for series in range(mY_.shape[2]):
        for n_set in range(mY_.shape[1]):
            index_consequents = np.where(mY_[:,n_set,series] > 0)

            index_premises = find_rules_by_consequent(complete_rules,series,n_set)
            if len(index_consequents) > 0: 
                if len(index_premises) > 0:
                    filter_prem = prem_terms[index_premises,:]

                    activated_prem = filter_prem[:,index_consequents[0]]

                    filtered_consequents = mY_[index_consequents,n_set,series]
                    tryout = 0
                    print('---------------------------')
                    print('Shape of activated prem is {}'.format(activated_prem.shape))
                    
                    cons = [{"type": "eq", "fun": constraint_function}]

                    bnds = [(0,1) for i in range(filter_prem.shape[0])]

                    res = minimize(objective_function,np.ones((activated_prem.shape[0])),args = (filtered_consequents,activated_prem), bounds = bnds, constraints = cons, tol = 1e-2)
                    print('Shape of initial guess is {}'.format(np.ones((activated_prem.shape[0])).shape))
                    print('Shape of res is {}'.format(res.x.shape))
                    print('Non-zeros weights = {}'.format(np.sum(np.where(res.x>0))))
                    
                    weighted_rules = activated_prem * res.x[:,None]
                    
                    aggr_rules = weighted_rules.max(axis=0)
                    
                    agg_training[index_consequents,n_set,series] = aggr_rules
                    
                    wd_[index_premises,n_set,series] = res.x
    isEmpty == False
            
            
            
            
            
            
            
            
            
            

# Defuzzification

In [ ]:
#Definition of support of every out.

def defuzz_cog(agg_training,mf_params_,num_series,show=False):
    y_predict_ = np.zeros((agg_training.shape[0],num_series))
    for i in range(num_series):

        a = int(mf_params_[-1,i] - mf_params_[0,i])
        support_discourse = np.linspace(mf_params_[0,i],mf_params_[-1,i],num=a)
        all_values = np.zeros((support_discourse.shape[0],mf_params_.shape[0]))

        for j in range(mf_params_.shape[0]):
            if j == 0:
                k = 0
                for val in support_discourse:
                    all_values[k,j] = trapmf(val,-1000*abs(mf_params_[j,i]),-1000*abs(mf_params_[j,i]),mf_params_[j,i],mf_params_[j+1,i])
                    k += 1
                #print(all_values[:,j,i])

            elif j < mf_params_.shape[0] - 1:
                k = 0
                for val in support_discourse:
                    all_values[k,j] = trimf(val,mf_params_[j-1,i],mf_params_[j,i],mf_params_[j+1,i])
                    k += 1

            else:
                k = 0
                for val in support_discourse:
                    all_values[k,j] = trapmf(val,mf_params_[j-1,i],mf_params_[j,i],1000*abs(mf_params_[j,i]),1000*abs(mf_params_[j,i]))
                    k += 1

        for p in range(agg_training.shape[0]):
            p_in = np.ones(shape=all_values.shape) * agg_training[p,:,i]  

            out = np.minimum(all_values,p_in)
            outResponse = np.maximum.reduce(out,axis=1)

            y_predict = sum(np.multiply(support_discourse,outResponse))/(sum(outResponse))

            y_predict_[p,i] = y_predict
            
        if show:
            plt.figure(figsize=(16,9))
            for i in range(all_values.shape[1]):
                plt.plot(support_discourse,out)
            plt.show()
            plt.close()
        
    return y_predict_

    
        

In [ ]:
def mape(A,F):
    return np.mean(np.abs(np.divide(A-F,A)))

In [ ]:
y_predict_ = defuzz_cog(agg_training,mf_params_,num_series,show=False)

for i in range(num_series):
    idx = np.where(np.isnan(y_predict_[:,i]))

    if len(idx) > 0:
        y_predict_[idx,i] = 0
        
        print('There are {} NaN in prediction'.format(len(idx[0])))
    print('MAE score for serie {} is {}'.format(i+1,mean_absolute_error(yt[:,i], y_predict_[:,i])))
    print('RMSE for serie {} is {}'.format(i+1,sqrt(mean_squared_error(yt[:,i], y_predict_[:,i]))))
    print('SMAPE for serie {} is {}'.format(i+1,smape(yt[:,i], y_predict_[:,i])))
    print('MAPE for serie {} is {}'.format(i+1,mape(yt[:,i], y_predict_[:,i])))
    print('R2 score for serie {} is {}'.format(i+1,r2_score(yt[:,i], y_predict_[:,i])))
    print('----------------------')
    

In [ ]:
for i in range(num_series):
    plt.figure(figsize=(10,6))
    plt.title('Serie {}'.format(i+1))
    plt.plot(y_predict_[:,i],color='blue')
    plt.plot(yt[:,i],color='red')
    plt.legend(['Predicted','Target'])

In [ ]:
if diff_series:
    y__ = y_predict_ + data[lag:in_sample.shape[0]-1,:]
    for i in range(num_series):

        compare_series = data[lag:in_sample.shape[0]-1,i]
        plt.figure(figsize=(10,6))

        plt.title('Serie {}'.format(i+1))

        plt.plot(y__[:,i],color='blue')

        plt.plot(data[lag:in_sample.shape[0]-1,i],color='red')

        plt.legend(['Predicted','Target'])

        print('MAE score for serie {} is {}'.format(i+1,mean_absolute_error(compare_series, y__[:,i])))
        print('RMSE for serie {} is {}'.format(i+1,sqrt(mean_squared_error(compare_series, y__[:,i]))))
        print('SMAPE for serie {} is {}'.format(i+1,smape(compare_series, y__[:,i])))
        print('MAPE for serie {} is {}'.format(i+1,mape(compare_series, y__[:,i])))
        print('R2 score for serie {} is {}'.format(i+1,r2_score(compare_series, y__[:,i])))
        print('----------------------')

# Trying to remove diff from time series

for i in range(num_series):
    plt.figure(figsize=(10,6))
    plt.title('Serie {}'.format(i+1))
    plt.plot(yt_without_diff[:,i],color='blue')
    plt.plot(in_sample[:,i],color='red')
    plt.legend(['Predicted','Target'])

In [ ]:
rulesize

In [ ]:
h_prev

# Prediction

In [ ]:
def find_rules_by_antecedent(rules,val):
    index_ = []
    k = 0
    for rule in rules:
        #print(rule)
        if check_if_inside(val,rule[0]):
            index_.append(k)
        k += 1
    #print(index_)
    return index_

In [ ]:
def test(rule,antecedents_activated):
    for term in rule[0]:
        
        if term in antecedents_activated:
            pass
        else:
            return False
    
    return True

In [ ]:
def blabla(a,b):
    for antec in a:
        print(antec)
        if not (antec in b):
            return False
    return True
            
    

In [ ]:
def prem_term(rule,muX):
    prem_concat = []
    for term in rule:
        #print(term)
        prem_concat.append(muX[0,term[1],term[0]])
    return tnorm_product(prem_concat)
        
        
        

In [ ]:
#Arrumação dos dados para previsão.
yp_totest = yp_lagged[yp_lagged.shape[0]-1:yp_lagged.shape[0],:]

In [ ]:
yt_totest = np.zeros((h_prev,num_series))

In [ ]:
yp_totest

In [ ]:
for h_p in range(h_prev):
    mX_values_in = np.zeros((1,mf_params_.shape[0],yp_totest.shape[1]))
    antecedents_activated = []
    for i in range(num_series):
        mf_params = mf_params_[:,i]
        for j in range(lag):

            mX, _ = Fuzzyfy.fuzzify(np.array([yp_totest[0,i*lag+j]]),mf_params)
            mX_values_in[:,:,i*lag+j] = mX


            idx_nonzero = np.where(mX[0,:] > 0)
            idx_nonzero = idx_nonzero[0]

            for k in range(idx_nonzero.shape[0]):
                antecedents_activated.append((i*lag+j,idx_nonzero[k]))
            

    check_idx = 0
    rules_idx = []
    prem_terms_test = np.zeros((rules.shape[0],1))

    for n_rule in rules:
        #print('Rule {} is {}'.format(check_idx,test(n_rule,antecedents_activated)))
        if test(n_rule,antecedents_activated):
            rules_idx.append(check_idx)
        check_idx += 1
        
    prem_activated = np.zeros((rules.shape[0],))
    for i in rules_idx:
        prem_activated[i,] = prem_term(rules[i,0],mX_values_in)
    
    agg_test = np.zeros((wd_.shape))
    for i in range(num_series):
        for j in rules_idx:
            rule = complete_rules[j,i]
            consequent = rule[-1]
            agg_test[j,consequent[1],i] = prem_activated[j,]
            
            
    weight_agg = np.multiply(agg_test,wd_)
    weight_ = np.zeros((weight_agg.shape[1],weight_agg.shape[2]))

    for i in range(weight_.shape[1]):
        weight_[:,i] = weight_agg[:,:,i].max(axis=0)

    w_todefuzz = np.reshape(weight_,(1,weight_.shape[0],weight_.shape[1]))
    
    
    y_pred = defuzz_cog(w_todefuzz,mf_params_,num_series,show=True)
    
    yt_totest[h_p,:] = y_pred
    
    yp_totest = np.roll(yp_totest,1)
    
    for i in range(num_series):
        yp_totest[0,i*lag] = y_pred[0][i]
    


In [ ]:
for i in range(num_series):
    print('RMSE for serie {} is {}'.format(i+1,sqrt(mean_squared_error(yt_totest[:,i], out_sample[:,i]))))
    print('SMAPE for serie {} is {}'.format(i+1,smape(out_sample[:,i],yt_totest[:,i])))

In [ ]:
for i in range(num_series):
    plt.figure(figsize=(10,6))
    plt.title('Serie {}'.format(i+1))
    plt.plot(yt_totest[:,i],color='blue')
    plt.plot(out_sample[:,i],color='red')
    plt.legend(['Predicted','Target'])

weight_agg = np.multiply(agg_test,wd_)
weight_ = np.zeros((weight_agg.shape[1],weight_agg.shape[2]))

for i in range(weight_.shape[1]):
    weight_[:,i] = weight_agg[:,:,i].max(axis=0)

w_todefuzz = np.reshape(weight_,(1,weight_.shape[0],weight_.shape[1]))

y_pred = defuzz_cog(w_todefuzz,mf_params_,num_series)
yt_totest[0,:] = y_pred

yt_totest[0,:] = y_pred

In [ ]:
yt_without_diff = np.zeros((yt_totest.shape))

In [ ]:
yt_without_diff[0,:] = in_sample[in_sample.shape[0]-1,:] + yt_totest[0,:]

In [ ]:
for i in range(1,h_prev):
    yt_without_diff[i,:] = yt_without_diff[i-1,:] + yt_totest[i,:]

In [ ]:
for i in range(num_series):
    print('RMSE for serie {} is {}'.format(i+1,sqrt(mean_squared_error(yt_without_diff[:,i], out_sample[:,i]))))

In [ ]:
out_without_diff = np.zeros((yt_totest.shape))

out_without_diff[0,:] = out_sample[0,:] - in_sample[in_sample.shape[0]-1,:]

for i in range(1,h_prev):
    out_without_diff[i,:] = out_sample[i,:] - out_sample[i-1,:]

In [ ]:
rulesize

In [ ]:
sum(wd_ > 0.1)

In [ ]:
rulesize